# RFC 001: Global run function

In [ ]:
from abc import ABC, abstractmethod
from typing import Annotated, Any, Iterable, Optional, Protocol, Union, runtime_checkable
from unittest.mock import MagicMock
from uuid import UUID

from pydantic import BaseModel, Field

from autogen import Agent

## Events

These are four different abstract types of events used by the runtime.

In [ ]:
@runtime_checkable
class EventProtocol(Protocol):
    @property
    def uuid(self) -> UUID: ...


@runtime_checkable
class InputRequestProtocol(EventProtocol, Protocol):
    @property
    def prompt(self) -> str: ...

    def respond(self, response: Union[str, "InputResponseProtocol"]) -> "InputResponseProtocol": ...


@runtime_checkable
class AsyncInputRequestProtocol(EventProtocol, Protocol):
    @property
    def prompt(self) -> str: ...

    async def respond(self, response: Union[str, "InputResponseProtocol"]) -> "InputResponseProtocol": ...


@runtime_checkable
class InputResponseProtocol(EventProtocol, Protocol):
    pass


@runtime_checkable
class OutputProtocol(EventProtocol, Protocol):
    @property
    def message(self) -> str: ...


@runtime_checkable
class SystemEventProtocol(EventProtocol, Protocol):
    pass

Helpers

In [ ]:
def is_input_request(event: EventProtocol) -> bool:
    return isinstance(event, InputRequestProtocol)


def is_async_input_request(event: EventProtocol) -> bool:
    return isinstance(event, AsyncInputRequestProtocol)


def is_input_response(event: EventProtocol) -> bool:
    return isinstance(event, InputResponseProtocol)


def is_output(event: EventProtocol) -> bool:
    return isinstance(event, OutputProtocol)


def is_system_event(event: EventProtocol) -> bool:
    return isinstance(event, SystemEventProtocol)

The actual implementation could use Pydantic `BaseModel`:

In [ ]:
class Event(BaseModel, ABC):
    uuid: Annotated[UUID, Field]


class InputRequest(Event, ABC):
    @property
    @abstractmethod
    def prompt(self) -> str: ...

    @abstractmethod
    def respond(self, response: "InputResponseProtocol"): ...


class InputResponse(Event, ABC):
    pass


class Output(Event, ABC):
    @property
    @abstractmethod
    def message(self) -> str: ...


class SystemEvent(Event, ABC):
    pass

## Client will abstract from the actual LLM used

In [ ]:
class LLMClientProtocol(Protocol):
    pass


class LLMMessageProtocol(Protocol):
    @property
    def client(self) -> LLMClientProtocol: ...

    @property
    def raw_message(self) -> Any: ...

## Global run function

In [ ]:
from typing import AsyncIterable


class RunResponseProtocol(Protocol):
    @property
    def events(self) -> Iterable[EventProtocol]: ...

    @property
    def messages(self) -> Iterable[LLMMessageProtocol]: ...

    @property
    def summary(self) -> str: ...


class AsyncRunResponseProtocol(Protocol):
    @property
    def events(self) -> AsyncIterable[EventProtocol]: ...

    @property
    def messages(self) -> AsyncIterable[LLMMessageProtocol]: ...

    @property
    async def summary(self) -> str: ...

In [ ]:
def run(
    *agents: Agent, message: Optional[str] = None, previous_run: Optional[RunResponseProtocol] = None, **kwargs: Any
) -> RunResponseProtocol:
    """Run the agents with the given initial message.

    Args:
        agents: The agents to run.
        message: The initial message to send to the first agent.
        previous_run: The previous run to continue.
        kwargs: Additional arguments to pass to the agents.

    """
    ...


async def a_run(
    *agents: Agent, message: Optional[str] = None, previous_run: Optional[RunResponseProtocol] = None, **kwargs: Any
) -> AsyncRunResponseProtocol:
    """Run the agents with the given initial message.

    Args:
        agents: The agents to run.
        message: The initial message to send to the first agent.
        previous_run: The previous run to continue.
        kwargs: Additional arguments to pass to the agents.

    """
    ...

## Event processing loop

In [ ]:
agents: list[Agent] = []

response = run(*agents, message="What is the meaning of life?")

response = MagicMock(spec=RunResponseProtocol)
response.events = [
    MagicMock(spec=InputRequestProtocol),
    MagicMock(spec=OutputProtocol),
    MagicMock(spec=SystemEventProtocol),
]

for m in response.events:
    if is_input_request(m):
        print(f"Input request: {m.prompt}")
        s = input(m.prompt)

        print(f"Response: {s}")
        m.respond(s)

    elif is_output(m):
        print(f"Output message: {m.message}")

    elif is_system_event(m):
        print(f"System event: {m}")

summary = response.summary

In [ ]:
agents: list[Agent] = []

response = await a_run(*agents, message="What is the meaning of life?")

response = MagicMock(spec=AsyncRunResponseProtocol)
events = [
    MagicMock(spec=AsyncInputRequestProtocol),
    MagicMock(spec=OutputProtocol),
    MagicMock(spec=SystemEventProtocol),
]


async def _events():
    for e in events:
        yield e


response.events = _events()

async for m in response.events:
    if is_input_request(m):
        print(f"Input request: {m.prompt}")
        s = input(m.prompt)

        print(f"Response: {s}")
        await m.respond(s)

    elif is_output(m):
        print(f"Output message: {m.message}")

    elif is_system_event(m):
        print(f"System event: {m}")

summary = response.summary

## Examples

In [ ]:
import autogen

config_list = autogen.config_list_from_json("../../notebook/OAI_CONFIG_LIST")

assert config_list

In [ ]:
class Cost:
    def __init__(self, **kwargs: Any):
        self._cost: dict[str, Any] = kwargs.copy()

    @staticmethod
    def _add_elements(key: str, x: dict[str, Any], y: dict[str, Any]) -> Any:
        if key in x and key in y:
            return x[key] + y[key]
        elif key in x:
            return x[key]
        elif key in y:
            return y[key]
        else:
            raise KeyError(f"Key {key} not found in either dictionary")

    def __add__(self, other: "Cost") -> "Cost":
        keys = set(self._cost.keys()) | set(other._cost.keys())
        return Cost(**{key: self._add_elements(key, self._cost, other._cost) for key in keys})